In [10]:
# ====================================================
# Library
# ====================================================
import sys
import os
import math
import time
import random
import shutil
from pathlib import Path
from contextlib import contextmanager
from collections import defaultdict, Counter

import scipy as sp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import preprocessing
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold, GroupKFold, KFold

from tqdm.auto import tqdm
from functools import partial

import cv2
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import Adam, SGD
import torchvision.models as models
from torch.nn.parameter import Parameter
from torch.utils.data import DataLoader, Dataset
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, CosineAnnealingLR, ReduceLROnPlateau
import timm

from albumentations import (
    Compose, OneOf, Normalize, Resize, RandomResizedCrop, RandomCrop,
    HorizontalFlip, VerticalFlip, RandomBrightnessContrast,
    Rotate, ShiftScaleRotate, CoarseDropout, Transpose  # Changed Cutout to CoarseDropout
)
from albumentations.pytorch import ToTensorV2

import warnings 
warnings.filterwarnings('ignore')

# Check available device
def get_device():
    if torch.backends.mps.is_available():
        return torch.device("mps")
    elif torch.cuda.is_available():
        return torch.device("cuda")
    else:
        return torch.device("cpu")

device = get_device()
print(f"Using device: {device}")

NOTEBOOK_DIR = Path(os.getcwd())  
PROJECT_ROOT = NOTEBOOK_DIR.parent  


INPUT_DIR = PROJECT_ROOT / 'input' / 'ranzcr-clip-catheter-line-classification'
TRAIN_PATH = INPUT_DIR / 'train'
TEST_PATH = INPUT_DIR / 'test'
OUTPUT_DIR = NOTEBOOK_DIR / 'output' 
os.makedirs(OUTPUT_DIR, exist_ok=True)

def verify_paths():
    paths = {
        'Project Root': PROJECT_ROOT,
        'Input Directory': INPUT_DIR,
        'Train Path': TRAIN_PATH,
        'Test Path': TEST_PATH,
        'Output Directory': OUTPUT_DIR
    }
    
    print("Checking paths:")
    for name, path in paths.items():
        exists = path.exists()
        print(f"{name}: {path}")
        print(f"Exists: {exists}")
        if not exists:
            print(f"WARNING: {name} does not exist!")
    
    
    if TRAIN_PATH.exists():
        train_images = list(TRAIN_PATH.glob('*.jpg'))[:5]  
        print(f"\nFirst few training images:")
        for img_path in train_images:
            print(f"Image exists: {img_path.name} - {img_path.exists()}")


verify_paths()


# ====================================================
# Config
# ====================================================
class CFG:
    debug = False
    print_freq = 100
    num_workers = 0
    model_name = 'resnext50_32x4d'
    size = 512
    scheduler = 'CosineAnnealingLR'
    epochs = 15
    T_max = 6
    lr = 1e-4
    min_lr = 1e-6
    batch_size = 4
    weight_decay = 1e-6
    gradient_accumulation_steps = 1
    max_grad_norm = 1000
    seed = 42
    target_size = 11
    target_cols = ['ETT - Abnormal', 'ETT - Borderline', 'ETT - Normal',
                   'NGT - Abnormal', 'NGT - Borderline', 'NGT - Incompletely Imaged', 'NGT - Normal',
                   'CVC - Abnormal', 'CVC - Borderline', 'CVC - Normal',
                   'Swan Ganz Catheter Present']
    n_fold = 4
    trn_fold = [0]
    train = True
    train_path = str(TRAIN_PATH)
    test_path = str(TEST_PATH)
    output_dir = str(OUTPUT_DIR)
    
  
    patience = 3
    early_stopping = True


def clear_memory():
    import gc
    gc.collect()
    if torch.backends.mps.is_available():
        torch.mps.empty_cache()

def seed_everything(seed=42):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.backends.mps.is_available():
        torch.mps.manual_seed(seed)

seed_everything(CFG.seed)

Using device: mps
Checking paths:
Project Root: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT
Exists: True
Input Directory: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification
Exists: True
Train Path: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification/train
Exists: True
Test Path: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification/test
Exists: True
Output Directory: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/notebooks/output
Exists: True

First few training images:
Image exists: 1.2.826.0.1.3680043.8.498.16451034714945708059993280774682419855.jpg - True
Image exists: 1.2.826.0.1.3680043.8.498.20326719114358003969350032771972492089.jpg - True
Image exists: 1.2.826.0.1.3680043.8.498.14

In [11]:
# ====================================================
# Dataset
# ====================================================
class TrainDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.labels = df[CFG.target_cols].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        try:
            file_name = self.file_names[idx]
            file_path = Path(CFG.train_path) / f'{file_name}.jpg'  
            
            if not file_path.exists():
                print(f"File not found: {file_path}")
                image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
            else:
                image = cv2.imread(str(file_path))
                if image is None:
                    print(f"Failed to read image: {file_path}")
                    image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
                else:
                    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            
            label = torch.tensor(self.labels[idx]).float()
            return image, label
            
        except Exception as e:
            print(f"Error loading image {file_name} at index {idx}: {str(e)}")
            image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
            if self.transform:
                augmented = self.transform(image=image)
                image = augmented['image']
            label = torch.tensor(self.labels[idx]).float()
            return image, label


class TestDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.file_names = df['StudyInstanceUID'].values
        self.transform = transform
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        file_name = self.file_names[idx]
        file_path = Path(CFG.test_path) / f'{file_name}.jpg'
        
        if not file_path.exists():
            print(f"Test file not found: {file_path}")
            image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
        else:
            image = cv2.imread(str(file_path))
            if image is None:
                print(f"Failed to read test image: {file_path}")
                image = np.zeros((CFG.size, CFG.size, 3), dtype=np.uint8)
            else:
                image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
                
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
            
        return image

In [12]:
# ====================================================
# Model Architecture
# ====================================================
class CustomResNext(nn.Module):
    def __init__(self, model_name='resnext50_32x4d', pretrained=True):
        super().__init__()
        self.model = timm.create_model(model_name, pretrained=pretrained)
        n_features = self.model.fc.in_features
        
        self.model.fc = nn.Sequential(
            nn.Dropout(0.5),
            nn.Linear(n_features, 512),
            nn.ReLU(),
            nn.BatchNorm1d(512),
            nn.Dropout(0.3),
            nn.Linear(512, CFG.target_size)
        )

    def forward(self, x):
        x = self.model(x)
        return x

In [13]:
# ====================================================
# Data Transforms
# ====================================================
def get_transforms(*, data):
    if data == 'train':
        return Compose([
            RandomResizedCrop(CFG.size, CFG.size, scale=(0.85, 1.0)),
            HorizontalFlip(p=0.5),
            RandomBrightnessContrast(p=0.5, brightness_limit=0.2, contrast_limit=0.2),
            ShiftScaleRotate(p=0.5, shift_limit=0.2, scale_limit=0.2, rotate_limit=20),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    elif data == 'valid':
        return Compose([
            Resize(CFG.size, CFG.size),
            Normalize(
                mean=[0.485, 0.456, 0.406],
                std=[0.229, 0.224, 0.225],
            ),
            ToTensorV2(),
        ])
    

In [14]:

# ====================================================
# Metrics
# ====================================================
def get_score(y_true, y_pred):
    scores = []
    for i in range(y_true.shape[1]):
        score = roc_auc_score(y_true[:, i], y_pred[:, i])
        scores.append(score)
    avg_score = np.mean(scores)
    return avg_score, scores

class AverageMeter(object):
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

def calculate_accuracy(outputs, targets):
    predictions = (outputs.sigmoid() > 0.5).float()
    correct = (predictions == targets).float().sum()
    total = targets.numel()
    return (correct / total).item()

# ====================================================
# Training Functions
# ====================================================
def train_one_epoch(model, loader, optimizer, criterion, device):
    model.train()
    train_loss = AverageMeter()
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc='Train')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        optimizer.zero_grad()
        
        y_preds = model(images)
        loss = criterion(y_preds, labels)
        loss.backward()
        
        grad_norm = torch.nn.utils.clip_grad_norm_(
            model.parameters(), 
            CFG.max_grad_norm
        )
        
        optimizer.step()
        
        train_loss.update(loss.item(), batch_size)
        
        if (step + 1) % CFG.print_freq == 0:
            pbar.set_postfix(**{
                'train_loss': train_loss.avg,
                'grad_norm': grad_norm.item(),
                'lr': optimizer.param_groups[0]['lr']
            })
        
        if step % 10 == 0:
            clear_memory()
            
    return train_loss.avg

def valid_one_epoch(model, loader, criterion, device):
    model.eval()
    valid_loss = AverageMeter()
    valid_acc = AverageMeter()  
    predictions = []
    targets = []
    
    pbar = tqdm(enumerate(loader), total=len(loader), desc='Valid')
    for step, (images, labels) in pbar:
        images = images.to(device)
        labels = labels.to(device)
        batch_size = labels.size(0)
        
        with torch.no_grad():
            y_preds = model(images)
            loss = criterion(y_preds, labels)
            
         
            acc = calculate_accuracy(y_preds, labels)
            valid_acc.update(acc, batch_size)
        
        predictions.append(y_preds.sigmoid().cpu().numpy())
        targets.append(labels.cpu().numpy())
        
        valid_loss.update(loss.item(), batch_size)
        
        if (step + 1) % CFG.print_freq == 0:
            pbar.set_postfix({
                'valid_loss': valid_loss.avg,
                'valid_acc': valid_acc.avg
            })
    
    predictions = np.concatenate(predictions)
    targets = np.concatenate(targets)
    
    return valid_loss.avg, valid_acc.avg, predictions, targets

In [15]:
def train_loop(train_loader, valid_loader, model, criterion, optimizer, scheduler, fold):
    best_score = 0.
    best_loss = np.inf
    patience_counter = 0
    
  
    log_file = f'{CFG.output_dir}/training_log_fold{fold}.csv'
    with open(log_file, 'w') as f:
        f.write('epoch,train_loss,train_acc,valid_loss,valid_acc,auc_score\n')
    
    for epoch in range(CFG.epochs):
        print(f'Epoch {epoch+1}/{CFG.epochs}')
        
    
        model.train()
        train_loss = AverageMeter()
        train_acc = AverageMeter()
        
        for images, labels in tqdm(train_loader, desc='Train'):
            images = images.to(device)
            labels = labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            acc = calculate_accuracy(outputs, labels)
            train_loss.update(loss.item(), labels.size(0))
            train_acc.update(acc, labels.size(0))
        
       
        valid_loss, valid_acc, predictions, targets = valid_one_epoch(model, valid_loader, criterion, device)
        score, scores = get_score(targets, predictions)
        

        if isinstance(scheduler, ReduceLROnPlateau):
            scheduler.step(valid_loss)
        else:
            scheduler.step()
        
      
        print(f'Train Loss: {train_loss.avg:.4f} Train Acc: {train_acc.avg:.4f}')
        print(f'Valid Loss: {valid_loss:.4f} Valid Acc: {valid_acc:.4f}')
        print(f'AUC Score: {score:.4f}')
        
      
        with open(log_file, 'a') as f:
            f.write(f'{epoch+1},{train_loss.avg:.4f},{train_acc.avg:.4f},'
                   f'{valid_loss:.4f},{valid_acc:.4f},{score:.4f}\n')
        
    
        if score > best_score:
            best_score = score
            best_loss = valid_loss
            patience_counter = 0
            print(f'Saving best model... Score: {best_score:.4f}')
            torch.save({
                'model': model.state_dict(),
                'predictions': predictions,
                'targets': targets,
                'epoch': epoch,
                'score': best_score
            }, f'{CFG.output_dir}/best_fold{fold}.pth')
        else:
            patience_counter += 1
            
      
        if CFG.early_stopping and patience_counter >= CFG.patience:
            print(f'Early stopping triggered after {epoch + 1} epochs')
            break
            
        clear_memory()
    
    return best_score, best_loss

In [16]:

# ====================================================
# Main Training Function
# ====================================================
def train_model():
    train_csv_path = INPUT_DIR / 'train.csv'
    test_csv_path = INPUT_DIR / 'sample_submission_copy.csv'
    
    print(f"Loading data from: {train_csv_path}")
    train = pd.read_csv(train_csv_path)
    test = pd.read_csv(test_csv_path)
    
    if CFG.debug:
        train = train.sample(n=1000, random_state=CFG.seed).reset_index(drop=True)
    
    
    Fold = GroupKFold(n_splits=CFG.n_fold)
    groups = train['PatientID'].values
    for n, (train_index, valid_index) in enumerate(Fold.split(train, train[CFG.target_cols], groups)):
        train.loc[valid_index, 'fold'] = int(n)
    train['fold'] = train['fold'].astype(int)
    
    for fold in CFG.trn_fold:
        print(f'Training fold {fold}')
        
        train_df = train[train.fold != fold].reset_index(drop=True)
        valid_df = train[train.fold == fold].reset_index(drop=True)
        
        train_dataset = TrainDataset(train_df, transform=get_transforms(data='train'))
        valid_dataset = TrainDataset(valid_df, transform=get_transforms(data='valid'))
        
        train_loader = DataLoader(
            train_dataset,
            batch_size=CFG.batch_size,
            shuffle=True,
            num_workers=CFG.num_workers,
            pin_memory=True,
            drop_last=True
        )
        
        valid_loader = DataLoader(
            valid_dataset,
            batch_size=CFG.batch_size,
            shuffle=False,
            num_workers=CFG.num_workers,
            pin_memory=True,
            drop_last=False
        )
        
        model = CustomResNext(CFG.model_name, pretrained=True)
        model.to(device)
        
        optimizer = Adam(model.parameters(), lr=CFG.lr, weight_decay=CFG.weight_decay)
        scheduler = CosineAnnealingLR(optimizer, T_max=CFG.T_max, eta_min=CFG.min_lr)
        criterion = nn.BCEWithLogitsLoss()
        
        best_score, best_loss = train_loop(
            train_loader,
            valid_loader,
            model,
            criterion,
            optimizer,
            scheduler,
            fold
        )
        
        print(f'Best Score: {best_score:.4f} Best Loss: {best_loss:.4f}')
        clear_memory()

In [17]:
def predict_test():
    print('Loading test data...')
    test_csv_path = INPUT_DIR / 'sample_submission_copy.csv'
    test_df = pd.read_csv(test_csv_path)
    
   
    test_dataset = TestDataset(
        test_df, 
        transform=get_transforms(data='valid')
    )
    test_loader = DataLoader(
        test_dataset,
        batch_size=CFG.batch_size,
        shuffle=False,
        num_workers=CFG.num_workers,
        pin_memory=True
    )
    
  
    models = []
    for fold in CFG.trn_fold:
        model = CustomResNext(CFG.model_name, pretrained=False)
        model_path = f'{OUTPUT_DIR}/best_fold{fold}.pth'
        model.load_state_dict(torch.load(model_path)['model'])
        model.to(device)
        model.eval()
        models.append(model)
    

    predictions = []
    with torch.no_grad():
        for images in tqdm(test_loader, desc='Predict'):
            images = images.to(device)
            outputs = None
            
        
            for model in models:
                if outputs is None:
                    outputs = model(images).sigmoid()
                else:
                    outputs += model(images).sigmoid()
            outputs /= len(models)
            
            predictions.append(outputs.cpu().numpy())
    
    predictions = np.concatenate(predictions)
    
   
    submission = pd.DataFrame(data=predictions, columns=CFG.target_cols)
    submission['StudyInstanceUID'] = test_df['StudyInstanceUID']
    submission.to_csv(INPUT_DIR / 'sample_submission_copy.csv', index=False)
    print('Predictions saved to sample_submission_copy.csv')

In [18]:
if __name__ == '__main__':
    print('Training started...')
    seed_everything(CFG.seed)
    train_model()
    print('Training completed!')
    
    print('\nGenerating test predictions...')
    predict_test()
    print('All done!')

Training started...
Loading data from: /Users/sankalpkashyap/Desktop/UCDavisStudy/ECS271-MLD/Project/CatheterPositionViT/input/ranzcr-clip-catheter-line-classification/train.csv
Training fold 0
Epoch 1/15


Train:   0%|          | 0/5640 [00:00<?, ?it/s]

Valid:   0%|          | 0/1881 [00:00<?, ?it/s]

Train Loss: 0.3306 Train Acc: 0.8552
Valid Loss: 0.2366 Valid Acc: 0.9013
AUC Score: 0.7761
Saving best model... Score: 0.7761
Epoch 2/15


Train:   0%|          | 0/5640 [00:00<?, ?it/s]

...

Train:   0%|          | 0/5640 [00:00<?, ?it/s]

Valid:   0%|          | 0/1881 [00:00<?, ?it/s]

Train Loss: 0.1892 Train Acc: 0.9196
Valid Loss: 0.1842 Valid Acc: 0.9189
AUC Score: 0.9086
Early stopping triggered after 11 epochs
Best Score: 0.9119 Best Loss: 0.1748
Training completed!

Generating test predictions...
Loading test data...


Predict:   0%|          | 0/896 [00:00<?, ?it/s]

Predictions saved to sample_submission_copy.csv
All done!
